In [71]:
from game import Game
import time
import os
import re

## Load maps

In [72]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [73]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [74]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [75]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [76]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [77]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [78]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [79]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

### BFS

In [ ]:
def solver_bfs(game_map, map_number=None, time_limit=None):
    from collections import deque
    import time
    
    start_time = time.time()
    game = Game(game_map)
    visited = set()
    queue = deque([(game, [])])
    visited_states = 0
    
    while queue:
        if time_limit and time.time() - start_time > time_limit:
            print(f"BFS time limit exceeded for map {map_number}")
            return None, visited_states
            
        current_game, moves = queue.popleft()
        visited_states += 1
        
        if current_game.is_game_won():
            return moves, visited_states
        
        player_pos = current_game.get_player_position()
        box_positions = tuple(sorted(current_game.get_box_locations()))
        state = (player_pos, box_positions)
        
        if state in visited:
            continue
            
        visited.add(state)

        for move in ['U', 'D', 'L', 'R']:
            new_game = Game(current_game.get_map())
            
            if new_game.apply_move(move):
                queue.append((new_game, moves + [move]))
    
    return None, visited_states

### DFS

In [ ]:
def solver_dfs(game_map, map_number=None, time_limit=None):
    import time
    
    start_time = time.time()
    game = Game(game_map)
    visited = set()
    stack = [(game, [])]
    visited_states = 0
    
    while stack:
        if time_limit and time.time() - start_time > time_limit:
            print(f"DFS time limit exceeded for map {map_number}")
            return None, visited_states
            
        current_game, moves = stack.pop()
        visited_states += 1
        
        if current_game.is_game_won():
            return moves, visited_states
        
        player_pos = current_game.get_player_position()
        box_positions = tuple(sorted(current_game.get_box_locations()))
        state = (player_pos, box_positions)
        
        if state in visited:
            continue
            
        visited.add(state)

        for move in ['R', 'L', 'D', 'U']:
            new_game = Game(current_game.get_map())
            
            if new_game.apply_move(move):
                stack.append((new_game, moves + [move]))
    
    return None, visited_states

### IDS

In [ ]:
def solver_ids(game_map, map_number=None, time_limit=None):
    import time
    
    start_time = time.time()
    game = Game(game_map)
    max_depth = 1
    total_visited_states = 0
    
    while True:
        if time_limit and time.time() - start_time > time_limit:
            print(f"IDS time limit exceeded for map {map_number}")
            return None, total_visited_states
            
        visited = set()
        result, visited_states = dfs_limited(game, max_depth, visited, time_limit=time_limit, 
                                            start_time=start_time, map_number=map_number)
        total_visited_states += visited_states
        
        if result is not None:
            return result, total_visited_states
            
        max_depth += 1
        if max_depth > 100: 
            break
    
    return None, total_visited_states

def dfs_limited(game, max_depth, visited, moves=None, depth=0, time_limit=None, start_time=None, map_number=None):
    import time
    
    if moves is None:
        moves = []
        
    if time_limit and start_time and time.time() - start_time > time_limit:
        return None, 0
        
    if depth > max_depth:
        return None, 0
        
    if game.is_game_won():
        return moves, 1
    
    player_pos = game.get_player_position()
    box_positions = tuple(sorted(game.get_box_locations()))
    state = (player_pos, box_positions)
    
    if state in visited:
        return None, 0
        
    visited.add(state)
    visited_states = 1
    
    for move in ['U', 'D', 'L', 'R']:
        new_game = Game(game.get_map())
        
        if new_game.apply_move(move):
            result, states = dfs_limited(new_game, max_depth, visited, moves + [move], depth + 1, 
                                        time_limit, start_time, map_number)
            visited_states += states
            if result is not None:
                return result, visited_states
    
    return None, visited_states

### A*

In [ ]:
def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def heuristic(game):
    total_distance = 0
    box_positions = game.get_box_locations()
    goal_positions = game.get_goal_locations()
    player_pos = game.get_player_position()
    
    for i in range(len(box_positions)):
        total_distance += manhattan_distance(box_positions[i], goal_positions[i])
    
    min_distance_to_box = float('inf')
    for i in range(len(box_positions)):
        if box_positions[i] != goal_positions[i]:
            dist = manhattan_distance(player_pos, box_positions[i])
            min_distance_to_box = min(min_distance_to_box, dist)
    
    if min_distance_to_box != float('inf'):
        total_distance += min_distance_to_box
        
    return total_distance

def solver_astar(game_map, heuristic_func=heuristic, weight=1, map_number=None, time_limit=None):
    import heapq
    import time
    
    start_time = time.time()
    game = Game(game_map)
    visited = set()
    counter = 0
    priority_queue = [(heuristic_func(game) * weight, counter, 0, game, [])]
    visited_states = 0
    
    while priority_queue:
        if time_limit and time.time() - start_time > time_limit:
            print(f"A* time limit exceeded for map {map_number}")
            return None, visited_states
            
        _, _, g_score, current_game, moves = heapq.heappop(priority_queue)
        visited_states += 1
        
        if current_game.is_game_won():
            return moves, visited_states
        
        player_pos = current_game.get_player_position()
        box_positions = tuple(sorted(current_game.get_box_locations()))
        state = (player_pos, box_positions)
        
        if state in visited:
            continue
            
        visited.add(state)
        
        for move in ['U', 'D', 'L', 'R']:
            new_game = Game(current_game.get_map())
            
            if new_game.apply_move(move):
                new_g_score = g_score + 1
                h_score = heuristic_func(new_game)
                f_score = new_g_score + h_score * weight
                counter += 1 
                heapq.heappush(priority_queue, (f_score, counter, new_g_score, new_game, moves + [move]))
    
    return None, visited_states

## Solve

In [84]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar
}

In [85]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [86]:
solve(1, "BFS")

BFS took 0.01 seconds on map 1 and visited 77 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']


In [87]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            solve(map, method)
            

In [ ]:
solve_all()

BFS took 0.01 seconds on map 1 and visited 77 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
DFS took 0.0 seconds on map 1 and visited 17 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
IDS took 0.03 seconds on map 1 and visited 155 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
A* took 0.01 seconds on map 1 and visited 51 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
BFS took 0.01 seconds on map 2 and visited 33 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
DFS took 0.0 seconds on map 2 and visited 11 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
IDS took 0.01 seconds on map 2 and visited 60 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
A* took 0.01 seconds on map 2 and visited 16 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
BFS took 0.03 seconds on map 3 and visited 249 states.
13 moves were used: ['U', 'L', 'D', 'D', 'U', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D']
DFS took 0

KeyboardInterrupt: 